**Plz Upvote if you like my Voting Ensemble approach**

* LeaderBoard Score - 85.300%

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Initializing Libraries

In [ ]:
import pandas as pd
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from PIL import Image

%matplotlib inline
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
import cv2
from tqdm import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Model
from keras import optimizers
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

## Getting the Data

In [ ]:
train = pd.read_csv('/kaggle/input/hackerearth-deep-learning-challenge-holidayseason/dataset/train.csv')
train.head()

In [ ]:
label_counts = train.Class.value_counts()
plt.figure(figsize = (18,10))
sns.barplot(label_counts.index, label_counts.values, alpha = 0.9)
plt.xticks(rotation = 'vertical')
plt.xlabel('Image Class', fontsize =12)
plt.ylabel('Counts', fontsize = 12)
plt.show()

## Taking out images for training

In [ ]:
TRAIN_PATH = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/train/'
TEST_PATH = '../input/hackerearth-deep-learning-challenge-holidayseason/dataset/test/'

In [ ]:
IMG_SIZE = 224

def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    return img

In [ ]:
train_img = []
for img_name in tqdm(train['Image'].values):
    train_img.append(read_img(TRAIN_PATH + img_name ))
    
x_train = np.array(train_img, np.float32) / 255
print(x_train.shape)

In [ ]:
label_list = train['Class'].tolist()
label_numeric = {k: v+1 for v, k in enumerate(set(label_list))}
y_train = [label_numeric[k] for k in label_list]
y_train = np.array(y_train)

In [ ]:
y_train = to_categorical(y_train)
print(y_train.shape)

In [ ]:
def callbacks(name):
    import keras
    checkpoint =keras.callbacks.ModelCheckpoint(filepath="HKE"+str(name)+".h5", 
                            monitor='accuracy',
                            verbose=1,
                            save_best_only=True, 
                            save_weights_only=False, 
                            mode='auto')
                            #save_freq='epoch')
        
    return [checkpoint]


## Model

VGG19

In [ ]:
base_model =   applications.VGG19(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dropout(0.3))
add_model.add(Dense(128, activation='relu'))
add_model.add(Dropout(0.5))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))

model.compile(loss='categorical_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

In [ ]:
batch_size = 8   
epochs = 15          

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1, 
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_datagen.fit(x_train)

In [ ]:
history = model.fit_generator( train_datagen.flow(x_train, y_train, batch_size=batch_size),
                               steps_per_epoch=x_train.shape[0] // batch_size,
                               epochs=epochs,callbacks=callbacks("vgg19"))

In [ ]:
import gc
gc.collect()

ResNet50

In [ ]:
base_model2 =   applications.NASNetMobile(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
for layers in base_model2.layers[:-5]:
    layers.trainable=False

add_model2 = Sequential()
add_model2.add(base_model2)
add_model2.add(Conv2D(64,(3,3),activation='relu'))
add_model2.add(Conv2D(32,(3,3),activation='relu'))
add_model2.add(Flatten())
add_model2.add(Dropout(0.3))
add_model2.add(Dense(512, activation='relu'))
add_model2.add(Dropout(0.5))
add_model2.add(Dense(y_train.shape[1], activation='softmax'))



add_model2.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

add_model2.summary()

In [ ]:
history2 = add_model2.fit_generator( train_datagen.flow(x_train, y_train, batch_size=batch_size),
                               steps_per_epoch=x_train.shape[0] // batch_size,
                               epochs=epochs,callbacks=callbacks("NasNet"))

Xception

In [ ]:
base_model3 =   applications.Xception(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
#for layers in base_model3.layers[:-3]:
#    layers.trainable=False
base_model3.trainable=False   
add_model3 = Sequential()
add_model3.add(base_model3)
add_model3.add(Flatten())
add_model3.add(Dropout(0.3))
add_model3.add(Dense(4083, activation='relu'))
add_model3.add(Dropout(0.5))
add_model3.add(Dense(y_train.shape[1], activation='softmax'))



add_model3.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

add_model3.summary()

In [ ]:
history3 = add_model3.fit_generator( train_datagen.flow(x_train, y_train, batch_size=batch_size),
                               steps_per_epoch=x_train.shape[0] // batch_size,
                               epochs=epochs,callbacks=callbacks("Xception"))

In [ ]:
del x_train, y_train ,train_img
import gc
gc.collect()

In [ ]:
import keras
model= keras.models.load_model('./HKEvgg19.h5')
#add_model3= keras.models.load_model('./HKEXception.h5')
add_model2=keras.models.load_model('./HKENasNet.h5')

## Test data prediction

In [ ]:
test_images = os.listdir(TEST_PATH)
test_img = []
for image in test_images:
    test_img.append(read_img(TEST_PATH + image))

Voting Ensemble

In [ ]:
import gc
gc.collect()

In [ ]:
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX) for model in members]
	yhats = np.array(yhats)
	# sum across ensemble members
	summed = np.sum(yhats, axis=0)
	# argmax across classes
	#result = np.argmax(summed, axis=1)
	return summed

In [ ]:
x_test = np.array(test_img, np.float32) / 255
predictions = ensemble_predictions([model,add_model2,add_model3],x_test)

In [ ]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in label_numeric.items()}
pred_labels = [rev_y[k] for k in predictions]

## Final File

In [ ]:
sub = pd.DataFrame({'Image': test_images, 'Class': pred_labels})
sub.to_csv('submission.csv', index = False)
sub.head()

In [ ]:
import gc
gc.collect()